## Importing data and Pre-processing

In [24]:
pip install pycaret -q

     |████████████████████████████████| 81kB 7.1MB/s 
ERROR: pandas-profiling 2.11.0 has requirement requests>=2.24.0, but you'll have requests 2.23.0 which is incompatible.


In [25]:
# Importing relevant packages

import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor 

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder, RobustScaler, StandardScaler
from sklearn.compose import ColumnTransformer
from imblearn.pipeline import make_pipeline, Pipeline

from pycaret.regression import *

%matplotlib inline

In [26]:
hitters = pd.read_excel('/content/MLB Contract Data Final Version (2) (1).xlsx')

In [27]:
hitters

,Year,Player,Pos'n,Age,Old Club,New Club,Years,Guarantee,Term,Player Agent,Club Owner,Baseball Ops head / club GM,Future WAR,Unnamed: 13,past WAR,HR,RBI,IBB,SB,AVG,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,wOBA,wRAA,wRC,RAR,Spd,wRC+,WPA,RE24,REW,Clutch
0,1999,"Alomar, Roberto",2b,31,BAL,CLE,4,32000000,1999-2002,Jaime Torres,Richard Jacobs,John Hart,20.3,5.075,7.3,28,116,5,27,0.303,0.088,0.100,0.88,0.365,0.452,0.817,0.149,0.316,0.358,24.6,164,74.8,4.5,115,-0.44,8.42,0.92,-1.62
1,1999,"Palmeiro, Rafael",1b,34,BAL,TEX,5,45000000,1999-2003,Bronner Gilhooley,Tom Hicks,Doug Melvin,20.0,4.000,10.2,81,231,15,16,0.275,0.104,0.143,0.73,0.355,0.526,0.880,0.251,0.268,0.376,51.4,224,103.3,3.4,127,4.45,47.41,4.57,0.28
2,1999,"Ventura, Robin",3b,31,CHA,NYN,4,32000000,1999-2002,John Boggs,Wilpon/Doubleday,Steve Phillips,15.7,3.925,7.6,27,117,20,1,0.263,0.126,0.148,0.86,0.355,0.433,0.788,0.171,0.284,0.341,6.9,117,77.0,3.0,104,3.06,11.46,1.05,3.00
3,1999,"Burks, Ellis",of,34,SFN,SFN,2,10000000,1999-2000,Jim Turner,Peter Magowan,Brian Sabean,7.3,3.650,3.3,53,158,1,18,0.291,0.099,0.175,0.56,0.364,0.530,0.894,0.239,0.310,0.385,46.6,177,33.7,5.1,114,1.56,33.07,3.30,-1.83
4,1999,"Finley, Steve",of,34,SDN,ARI,5,21500000,1999-2003,Tommy Tanzer,Jerry Colangelo,Joe Garagiola Jr.,17.0,3.400,0.8,42,159,2,27,0.254,0.068,0.151,0.45,0.307,0.436,0.743,0.182,0.271,0.323,-10.0,149,8.8,6.3,97,0.31,7.01,0.54,-0.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879,2019,"Kang, Jung Ho",3b,32,PIT,PIT,1,3000000,2019,Wasserman,Bob Nutting,Neal Huntington,-0.9,-0.900,0.0,0,0,0,0,0.333,0.000,0.167,0.00,0.333,0.333,0.667,0.000,0.400,0.293,-0.1,1,-0.1,0.1,83,-0.01,0.11,0.01,0.03
880,2019,"Maldonado, Martin",c,32,HOU,KCA,1,2500000,2019,Magnus Sports,David Glass,Dayton Moore,0.8,0.800,5.0,23,82,1,0,0.223,0.035,0.248,0.14,0.276,0.360,0.637,0.137,0.276,0.277,-29.8,75,49.3,2.4,72,-5.34,-36.78,-3.85,-2.55
881,2019,"McCann, Brian",c,35,HOU,ATL,1,2000000,2019,Jet Sports,Liberty Media,Alex Anthopoulos,1.1,1.100,2.7,25,85,3,1,0.230,0.093,0.159,0.58,0.315,0.401,0.717,0.171,0.235,0.310,-4.5,69,27.5,2.3,96,0.14,-9.64,-0.94,-0.04
882,2019,"Harrison, Josh",2b-3b,31,PIT,DET,1,2000000,2019,MSM Sports,Chris Ilitch,Al Avila,-0.7,-0.700,2.9,24,84,3,15,0.263,0.050,0.172,0.29,0.320,0.404,0.724,0.141,0.296,0.313,-4.1,106,29.2,4.7,93,-0.84,-1.62,-0.14,-0.21


In [28]:
hitters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 884 entries, 0 to 883
Data columns (total 38 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Year                         884 non-null    int64  
 1   Player                       884 non-null    object 
 2   Pos'n                        884 non-null    object 
 3   Age                          884 non-null    int64  
 4   Old Club                     882 non-null    object 
 5   New Club                     884 non-null    object 
 6   Years                        884 non-null    int64  
 7   Guarantee                    884 non-null    int64  
 8   Term                         884 non-null    object 
 9   Player Agent                 768 non-null    object 
 10  Club Owner                   882 non-null    object 
 11  Baseball Ops head / club GM  882 non-null    object 
 12  Future WAR                   884 non-null    float64
 13  Unnamed: 13         

In [29]:
# Changing the name of the position column to avoid python issues

hitters.rename(columns={'Pos\'n':'Position'}, inplace=True)

# Filling the null values of the dataframe

for column in hitters.select_dtypes('object'):
  hitters[column].fillna('unknown', inplace=True)

hitters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 884 entries, 0 to 883
Data columns (total 38 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Year                         884 non-null    int64  
 1   Player                       884 non-null    object 
 2   Position                     884 non-null    object 
 3   Age                          884 non-null    int64  
 4   Old Club                     884 non-null    object 
 5   New Club                     884 non-null    object 
 6   Years                        884 non-null    int64  
 7   Guarantee                    884 non-null    int64  
 8   Term                         884 non-null    object 
 9   Player Agent                 884 non-null    object 
 10  Club Owner                   884 non-null    object 
 11  Baseball Ops head / club GM  884 non-null    object 
 12  Future WAR                   884 non-null    float64
 13  Unnamed: 13         

In [30]:
# Checking the number of unique players in the dataframe
print(hitters['Player'].nunique())

492


## Exploratory Data Analysis

In [31]:
hitters.columns

Index(['Year', 'Player', 'Position', 'Age', 'Old Club', 'New Club', 'Years',
       'Guarantee', 'Term', 'Player Agent', 'Club Owner',
       'Baseball Ops head / club GM', 'Future WAR', 'Unnamed: 13', 'past WAR',
       'HR', 'RBI', 'IBB', 'SB', 'AVG', 'BB%', 'K%', 'BB/K', 'OBP', 'SLG',
       'OPS', 'ISO', 'BABIP', 'wOBA', 'wRAA', 'wRC', 'RAR', 'Spd', 'wRC+',
       'WPA', 'RE24', 'REW', 'Clutch'],
      dtype='object')

In [32]:
#@title Histogram Plot

feature = 'Guarantee' #@param ['Year', 'Player', 'Position', 'Age', 'Old Club', 'New Club', 'Years','Guarantee', 'Term', 'Player Agent', 'Club Owner','Baseball Ops head / club GM', 'Future WAR', 'Unnamed: 13', 'past WAR','HR', 'RBI', 'IBB', 'SB', 'AVG', 'BB%', 'K%', 'BB/K', 'OBP', 'SLG','OPS', 'ISO', 'BABIP', 'wOBA', 'wRAA', 'wRC', 'RAR', 'Spd', 'wRC+','WPA', 'RE24', 'REW', 'Clutch']
split_by = 'Year' #@param ['Year', 'Player', 'Position', 'Age', 'Old Club', 'New Club', 'Years','Guarantee', 'Term', 'Player Agent', 'Club Owner','Baseball Ops head / club GM', 'Future WAR', 'Unnamed: 13', 'past WAR','HR', 'RBI', 'IBB', 'SB', 'AVG', 'BB%', 'K%', 'BB/K', 'OBP', 'SLG','OPS', 'ISO', 'BABIP', 'wOBA', 'wRAA', 'wRC', 'RAR', 'Spd', 'wRC+','WPA', 'RE24', 'REW', 'Clutch']

# Histogram of contracts

px.histogram(hitters.sort_values(by=split_by),
             x= feature,
             marginal='box',
             color=split_by)

In [33]:
#@title Mean Contract Value

# Checking Players / Agents / Club Owners associated with high value contracts

feature = 'Guarantee' #@param ['Year', 'Player', 'Position', 'Age', 'Old Club', 'New Club', 'Years','Guarantee', 'Term', 'Player Agent', 'Club Owner','Baseball Ops head / club GM', 'Future WAR', 'Unnamed: 13', 'past WAR','HR', 'RBI', 'IBB', 'SB', 'AVG', 'BB%', 'K%', 'BB/K', 'OBP', 'SLG','OPS', 'ISO', 'BABIP', 'wOBA', 'wRAA', 'wRC', 'RAR', 'Spd', 'wRC+','WPA', 'RE24', 'REW', 'Clutch']
split_by = "Player" #@param ['Year', 'Player', 'Position', 'Age', 'Old Club', 'New Club', 'Years','Guarantee', 'Term', 'Player Agent', 'Club Owner','Baseball Ops head / club GM', 'Future WAR', 'Unnamed: 13', 'past WAR','HR', 'RBI', 'IBB', 'SB', 'AVG', 'BB%', 'K%', 'BB/K', 'OBP', 'SLG','OPS', 'ISO', 'BABIP', 'wOBA', 'wRAA', 'wRC', 'RAR', 'Spd', 'wRC+','WPA', 'RE24', 'REW', 'Clutch']

# Getting the mean value of AAV

hitters_mean = hitters.groupby([split_by])[feature].mean().reset_index()
hitters_mean.sort_values(by=feature, ascending=True, inplace=True)

# Exporting the data to csv file

hitters_mean.to_csv('hitters_mean.csv')

# Bar plots

px.bar(hitters_mean,
       x= split_by,
       y= feature,
      #  hover_data=['Player', 'Year', 'Guarantee', 'Years']
       )

In [51]:
feature = 'Guarantee' #@param ['Year', 'Player', 'Position', 'Age', 'Old Club', 'New Club', 'Years','Guarantee', 'Term', 'Player Agent', 'Club Owner','Baseball Ops head / club GM', 'Future WAR', 'Unnamed: 13', 'past WAR','HR', 'RBI', 'IBB', 'SB', 'AVG', 'BB%', 'K%', 'BB/K', 'OBP', 'SLG','OPS', 'ISO', 'BABIP', 'wOBA', 'wRAA', 'wRC', 'RAR', 'Spd', 'wRC+','WPA', 'RE24', 'REW', 'Clutch']
split_by = "Baseball Ops head / club GM" #@param ['Year', 'Player', 'Position', 'Age', 'Old Club', 'New Club', 'Years','Guarantee', 'Term', 'Player Agent', 'Club Owner','Baseball Ops head / club GM', 'Future WAR', 'Unnamed: 13', 'past WAR','HR', 'RBI', 'IBB', 'SB', 'AVG', 'BB%', 'K%', 'BB/K', 'OBP', 'SLG','OPS', 'ISO', 'BABIP', 'wOBA', 'wRAA', 'wRC', 'RAR', 'Spd', 'wRC+','WPA', 'RE24', 'REW', 'Clutch']

# hitters.sort_values(by=feature, ascending=True, inplace=True)

fig = px.box(hitters,
       x= split_by,
       y= feature,
       hover_data=['Player', 'Year', 'Guarantee', 'Position']
       )

fig.show()

# hitters.sort_values(by='Year', ascending=True, inplace=True)

In [35]:
correlation_years = pd.DataFrame()

for year in hitters['Year'].unique():
  hitters_corr = pd.DataFrame(hitters[hitters['Year'] == year].corr())
  hitters_corr = pd.DataFrame(hitters_corr['Guarantee'])
  correlation_years[year] = hitters_corr['Guarantee']

correlation_years = correlation_years.transpose()

correlation_years

,Year,Age,Years,Guarantee,Future WAR,Unnamed: 13,past WAR,HR,RBI,IBB,SB,AVG,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,wOBA,wRAA,wRC,RAR,Spd,wRC+,WPA,RE24,REW,Clutch
1999,NaN,-0.342099,0.915509,1.0,0.626074,0.311246,0.809928,0.637246,0.699337,0.636346,0.073323,0.507399,0.187479,-0.051743,0.157884,0.463601,0.649828,0.669933,0.534503,0.281107,0.634415,0.749821,0.738878,0.811750,0.079444,0.630891,0.545204,0.646741,0.637458,-0.254659
2000,NaN,-0.199545,0.838472,1.0,0.536491,0.355096,0.748215,0.824683,0.798337,0.462568,0.192755,0.042881,0.215936,0.128887,0.174399,0.202986,0.433704,0.414757,0.493516,-0.230846,0.388584,0.636070,0.769393,0.750420,0.072672,0.476392,0.642554,0.645914,0.664716,-0.159722
2001,NaN,-0.562004,0.914147,1.0,0.972026,0.587817,0.780388,0.687197,0.637741,0.361756,0.236754,0.353745,0.267383,0.187435,-0.015935,0.459545,0.586312,0.598425,0.543194,0.257394,0.580702,0.715580,0.624350,0.785734,0.075451,0.588862,0.602955,0.674334,0.662148,-0.228776
2002,NaN,-0.096087,0.882475,1.0,0.853660,0.586610,0.891242,0.796129,0.712807,0.834487,-0.046094,0.678373,0.712362,-0.134412,0.654283,0.818104,0.785548,0.843267,0.727707,0.213262,0.837446,0.913858,0.840019,0.892687,-0.138243,0.857341,0.890370,0.933641,0.932537,0.010598
2003,NaN,-0.278662,0.885719,1.0,0.799561,0.324854,0.683537,0.716389,0.580380,0.589618,0.031564,0.315628,0.438472,0.188453,0.141965,0.563701,0.624177,0.662481,0.600315,0.198842,0.645694,0.757947,0.663414,0.683051,-0.021223,0.686105,0.673970,0.687997,0.673206,-0.089053
2004,NaN,-0.354269,0.913394,1.0,0.909045,0.563495,0.708367,0.510095,0.565641,0.571947,0.347055,0.585564,0.090013,-0.239223,0.293136,0.474106,0.567886,0.596805,0.378025,0.289849,0.580602,0.690395,0.622659,0.708948,0.189992,0.593816,0.705393,0.732490,0.734674,0.168267
2005,NaN,-0.486403,0.919206,1.0,0.914022,0.566836,0.710363,0.578017,0.586368,0.380630,0.507568,0.413137,0.360370,-0.045271,0.285837,0.506668,0.576796,0.602141,0.493503,0.134546,0.590811,0.659684,0.625452,0.710222,0.299609,0.606345,0.644203,0.658037,0.654146,0.122982
2006,NaN,-0.427141,0.900086,1.0,0.851657,0.396284,0.660211,0.576698,0.622706,0.310064,0.509425,0.343468,0.120617,-0.159496,0.261736,0.313384,0.392930,0.407508,0.250593,0.116703,0.393056,0.596873,0.685087,0.657633,0.256393,0.406372,0.558607,0.597654,0.595496,0.054507
2007,NaN,-0.360005,0.916382,1.0,0.867005,0.382350,0.585380,0.678049,0.647378,0.321232,0.428338,0.212421,0.038296,0.006229,0.023523,0.149751,0.487308,0.415430,0.444131,-0.023886,0.376609,0.512326,0.665911,0.591573,0.179878,0.366410,0.445148,0.534586,0.529137,-0.220640
2008,NaN,-0.185431,0.936435,1.0,0.836230,0.219585,0.770703,0.781037,0.738921,0.500009,0.476912,0.299606,0.313906,0.066115,0.059850,0.444576,0.532350,0.536251,0.483089,0.243597,0.522054,0.758997,0.674340,0.770542,0.196373,0.525191,0.677648,0.761822,0.754057,-0.092103


In [36]:
correlation_years.columns

Index(['Year', 'Age', 'Years', 'Guarantee', 'Future WAR', 'Unnamed: 13',
       'past WAR', 'HR', 'RBI', 'IBB', 'SB', 'AVG', 'BB%', 'K%', 'BB/K', 'OBP',
       'SLG', 'OPS', 'ISO', 'BABIP', 'wOBA', 'wRAA', 'wRC', 'RAR', 'Spd',
       'wRC+', 'WPA', 'RE24', 'REW', 'Clutch'],
      dtype='object')

In [57]:
feature = 'wRAA' #@param ['Year', 'Age', 'Years', 'Guarantee', 'Future WAR', 'Unnamed: 13','past WAR', 'HR', 'RBI', 'IBB', 'SB', 'AVG', 'BB%', 'K%', 'BB/K', 'OBP','SLG', 'OPS', 'ISO', 'BABIP', 'wOBA', 'wRAA', 'wRC', 'RAR', 'Spd','wRC+', 'WPA', 'RE24', 'REW', 'Clutch']

px.bar(correlation_years,
       x=correlation_years.index,
       y=feature)

## Regression Model



In [77]:
#@title Scatter Plot

y = 'Guarantee' #@param ['Year', 'Player', 'Position', 'Age', 'Old Club', 'New Club', 'Years','Guarantee', 'Term', 'Player Agent', 'Club Owner','Baseball Ops head / club GM', 'Future WAR', 'Unnamed: 13', 'past WAR','HR', 'RBI', 'IBB', 'SB', 'AVG', 'BB%', 'K%', 'BB/K', 'OBP', 'SLG','OPS', 'ISO', 'BABIP', 'wOBA', 'wRAA', 'wRC', 'RAR', 'Spd', 'wRC+','WPA', 'RE24', 'REW', 'Clutch']
x = 'past WAR' #@param ['Year', 'Player', 'Position', 'Age', 'Old Club', 'New Club', 'Years','Guarantee', 'Term', 'Player Agent', 'Club Owner','Baseball Ops head / club GM', 'Future WAR', 'Unnamed: 13', 'past WAR','HR', 'RBI', 'IBB', 'SB', 'AVG', 'BB%', 'K%', 'BB/K', 'OBP', 'SLG','OPS', 'ISO', 'BABIP', 'wOBA', 'wRAA', 'wRC', 'RAR', 'Spd', 'wRC+','WPA', 'RE24', 'REW', 'Clutch']

# Scatter plots

px.scatter(hitters,
       x= x,
       y= y,
       color='Year',
       hover_data=['Player', 'Guarantee', 'Position']
       )

In [66]:
# Splitting data into X and Y

X = hitters.drop(columns=['Guarantee'])
y = hitters['Guarantee']

# Dropping non-relevant features

features_drop = ['Year', 'Years', 'Term', 'Player', 'Position', 'New Club', 'Player Agent', 'Club Owner', 'Old Club', 'Baseball Ops head / club GM', 'Unnamed: 13']

X.drop(columns=features_drop, inplace=True)

In [67]:
X

,Age,Guarantee,past WAR,HR,RBI,IBB,SB,AVG,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,wOBA,wRAA,wRC,RAR,Spd,wRC+,WPA,RE24,REW,Clutch
0,31,32000000,7.3,28,116,5,27,0.303,0.088,0.100,0.88,0.365,0.452,0.817,0.149,0.316,0.358,24.6,164,74.8,4.5,115,-0.44,8.42,0.92,-1.62
1,34,45000000,10.2,81,231,15,16,0.275,0.104,0.143,0.73,0.355,0.526,0.880,0.251,0.268,0.376,51.4,224,103.3,3.4,127,4.45,47.41,4.57,0.28
2,31,32000000,7.6,27,117,20,1,0.263,0.126,0.148,0.86,0.355,0.433,0.788,0.171,0.284,0.341,6.9,117,77.0,3.0,104,3.06,11.46,1.05,3.00
3,34,10000000,3.3,53,158,1,18,0.291,0.099,0.175,0.56,0.364,0.530,0.894,0.239,0.310,0.385,46.6,177,33.7,5.1,114,1.56,33.07,3.30,-1.83
4,34,21500000,0.8,42,159,2,27,0.254,0.068,0.151,0.45,0.307,0.436,0.743,0.182,0.271,0.323,-10.0,149,8.8,6.3,97,0.31,7.01,0.54,-0.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879,32,3000000,0.0,0,0,0,0,0.333,0.000,0.167,0.00,0.333,0.333,0.667,0.000,0.400,0.293,-0.1,1,-0.1,0.1,83,-0.01,0.11,0.01,0.03
880,32,2500000,5.0,23,82,1,0,0.223,0.035,0.248,0.14,0.276,0.360,0.637,0.137,0.276,0.277,-29.8,75,49.3,2.4,72,-5.34,-36.78,-3.85,-2.55
881,35,2000000,2.7,25,85,3,1,0.230,0.093,0.159,0.58,0.315,0.401,0.717,0.171,0.235,0.310,-4.5,69,27.5,2.3,96,0.14,-9.64,-0.94,-0.04
882,31,2000000,2.9,24,84,3,15,0.263,0.050,0.172,0.29,0.320,0.404,0.724,0.141,0.296,0.313,-4.1,106,29.2,4.7,93,-0.84,-1.62,-0.14,-0.21


In [68]:
# One hot encoding categorical features using get_dummies

X_encoded = pd.get_dummies(X, drop_first=True)
# X_encoded = X.copy()

# Filling missing values with median values

for column in X_encoded.columns:
  median = X_encoded[column].median()
  X_encoded[column].fillna(median, inplace=True)

# Creating Train and test set

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=0)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(707, 26)
(177, 26)
(707,)
(177,)


In [69]:
# Normalizing the data

for column in X_train.columns:
  X_train[column] = (X_train[column] - X_train[column].mean()) / X_train[column].std()

In [70]:
X_train

,Age,Guarantee,past WAR,HR,RBI,IBB,SB,AVG,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,wOBA,wRAA,wRC,RAR,Spd,wRC+,WPA,RE24,REW,Clutch
337,-0.513597,0.039904,-0.632460,1.032872,0.763239,0.846394,-0.630066,-0.876739,-0.099145,0.860821,-0.655350,-0.662413,0.157227,-0.120941,0.606207,-0.808328,-0.367429,-0.341314,0.394197,-0.619157,-1.223304,-0.396551,-0.006801,-0.287007,-0.337013,0.523907
253,1.126481,0.574473,1.426207,1.469912,1.101677,3.103293,-0.630066,1.801460,1.540700,-0.618054,1.661122,2.308420,2.394190,2.590393,1.886974,0.824991,2.541591,2.949339,1.585374,1.311002,-1.100879,2.822778,1.499015,2.069977,2.060071,-1.418672
686,-0.841613,-0.371302,0.030500,0.158790,0.325260,0.362773,0.166158,-0.281583,-0.163452,0.686836,-0.617375,-0.339496,-0.202564,-0.270929,-0.097314,0.303719,-0.217480,-0.151151,0.259709,-0.023130,0.429442,0.070018,-0.478088,0.251701,0.284591,-0.753878
270,-0.185582,-0.289061,-0.283534,0.104160,-0.391433,-0.604469,-0.140082,-0.318781,-1.063760,0.756430,-1.149024,-1.082205,1.127099,0.444401,1.454039,-0.495564,0.322338,0.183702,-0.662493,-0.317678,1.102783,0.816530,0.567938,0.249123,0.280869,0.808819
656,1.126481,-0.319901,0.414320,1.306022,1.460023,0.201566,-0.691314,0.387966,1.058393,-0.217888,0.749723,0.984462,0.939382,1.044356,0.894830,-0.252304,1.162055,1.341218,1.143486,0.392704,-1.345730,1.283099,1.230804,1.605282,1.695297,0.558442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,-1.169628,-0.412422,-0.911602,-0.769921,-1.526196,-0.926883,-0.691314,-2.290232,3.019776,2.861652,-0.085726,0.500087,-1.469654,-0.917035,-0.584366,0.373222,-0.547369,-0.320644,-1.123594,-0.896379,-1.590581,-0.256580,-0.451267,-0.555440,-0.571510,-0.848849
192,-1.497644,0.327749,0.274749,2.999555,2.097083,2.458464,-0.017586,-0.504767,2.762546,1.469770,0.445924,1.694879,1.721537,1.875063,2.229715,-0.460813,1.851823,2.465663,2.219388,0.292211,-0.305112,1.516384,1.763395,2.141780,2.127070,-1.151028
629,-0.841613,-0.361022,0.135178,-0.114360,-0.112719,-0.604469,-0.446322,0.239177,-0.902990,0.251873,-0.921174,-0.630121,0.360588,0.040586,0.299544,0.442725,-0.007550,-0.250366,-0.239817,0.143204,0.000952,-0.116609,-0.071939,0.033346,0.053816,-0.114986
559,0.798465,-0.463823,-0.946494,-0.715291,-1.147941,-0.926883,-0.568818,-2.104246,-0.195606,2.270102,-1.111049,-1.566580,-0.922146,-1.240087,-0.043197,-0.877830,-1.237136,-0.771249,-1.450207,-0.958754,-0.794814,-1.516318,-0.485751,-0.663329,-0.664564,0.757017


In [71]:
# Applying constant and creating the model

X_train_sm = sm.add_constant(X_train)

model = sm.OLS(y_train, X_train_sm).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             Future WAR   R-squared:                       0.664
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     51.70
Date:                Tue, 20 Apr 2021   Prob (F-statistic):          4.02e-142
Time:                        14:35:01   Log-Likelihood:                -1754.8
No. Observations:                 707   AIC:                             3564.
Df Residuals:                     680   BIC:                             3687.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.0492      0.111     18.455      0.0

In [74]:
# Dropping statistically insignificant variables

features_drop = ['REW', 'OBP', 'OPS', 'wOBA', 'BB%', 'HR', 'WPA', 'AVG', 'Clutch', 'RE24', 'IBB', 'BB/K', 'SLG', 'ISO', 'K%']

X_train_new = X_train.drop(columns=features_drop)

# Applying a constant and creating the model

X_train_new_sm = sm.add_constant(X_train_new)

model = sm.OLS(y_train, X_train_new_sm).fit()

print(model.summary())


                            OLS Regression Results                            
Dep. Variable:             Future WAR   R-squared:                       0.642
Model:                            OLS   Adj. R-squared:                  0.638
Method:                 Least Squares   F-statistic:                     179.0
Date:                Tue, 20 Apr 2021   Prob (F-statistic):          2.97e-151
Time:                        14:43:50   Log-Likelihood:                -1777.4
No. Observations:                 707   AIC:                             3571.
Df Residuals:                     699   BIC:                             3607.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.0492      0.113     18.124      0.0

### Regression with Pycaret

In [ ]:
hitters.columns

In [ ]:
# Preparing the training data

setup(hitters, 
      target='Guarantee',
      # numeric_features = ['Years', 'HR', 'HBP', 'WP', 'W'],
      numeric_features = ['Age'],
      ignore_features = ['Player',
                         'Years',
                         'Unnamed: 13',
                         'Term'],
      normalize = True, 
      normalize_method = 'zscore',
      )

In [ ]:
compare_models()

In [ ]:
model = create_model('gbr')

In [ ]:
plot_model(model, plot = 'feature')

In [ ]:
evaluate_model(model)